In [ ]:
# contributors: Ari Alavi, John Kirchner, Gokul Deep
import os
import json
from os.path import exists, join, basename, splitext

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

class GoogleDriveDatabase:
  def __init__(self, drive, DATABASE_GID:str):
    assert isinstance(DATABASE_GID, str)
    self.folders = {}
    self.drive = drive
    self.database_gid = DATABASE_GID
    folder_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(DATABASE_GID)}).GetList()
    for file in folder_list:
      if file['mimeType'] == "application/vnd.google-apps.folder":
        self.folders[file['title']] = file['id']
    self.createFolder("TRASH")
    print("{} folders loaded".format(len(self.folders.keys())))
  def upload(self, filename, character, fileType):
    assert isinstance(filename, str)
    assert isinstance(character, str)
    assert isinstance(fileType, str)
    FILETYPE_MIME_MAP = {
        "jpeg" : "image/jpeg",
        "json" : "application/json",
        "zip" : "application/zip"
    }
    assert fileType in FILETYPE_MIME_MAP.keys(), "fileType must be one of the following: {}".format(fileType)
    assert os.path.isfile(filename), "{} does not exist as a file".format(filename)
    assert self.checkFolder(character), "{} is not a valid character. Pick from list: \n{}".format(character, tuple(self.folders.keys()))
    file = self.drive.CreateFile({
        "title" :  os.path.split(filename)[1],
        "mimeType" : FILETYPE_MIME_MAP[fileType],
        "parents" : [{"id" : self.folders[character]}]
    })
    file.SetContentFile(filename)
    file.Upload()
    os.remove(filename)
    print("uploaded and deleted {}".format(filename))

  @staticmethod
  def allImagesToJPG(fileName) -> str:
    assert isinstance(fileName, str)
    assert any(extension for extension in GoogleDriveDatabase.IMAGE_EXTENSIONS()), "{} is not a valid image".format(fileName)
    if ".jpg" in fileName:
      return fileName
    return fileName.split(".")[0] + ".jpg"
    
  @staticmethod
  def IMAGE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png"]

  @staticmethod
  def FILE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png", ".zip", ".json"]

  def getFiles(self, character) -> list:
    return [
      x for x in self.drive.ListFile({'q': "'{}' in parents and trashed=false".format(self.folders[character])}).GetList()
      if x['mimeType'] != "application/vnd.google-apps.folder" 
    ]
  def download_file_name(self, file_name):
      if not any(extension in file_name for extension in GoogleDriveDatabase.FILE_EXTENSIONS()):
        file_name += ".jpg"
      return GoogleDriveDatabase.allImagesToJPG(file_name)

  def download_file(self, file, folder:str, **kwargs) -> str:
      check_already_exist = kwargs.get("check_local", False)
      file_name = os.path.join(folder, file['title'])
      file_name = self.download_file_name(file_name)
      if check_already_exist:
        local_files = [os.path.join(folder, file) for file in os.listdir(folder)]
        if file_name in local_files:
          return file_name
      file.GetContentFile(file_name)
      print("downloaded", file_name)
      return file_name

  def download(self, character:str,folder:str):
    file_list = self.getFiles(character)
    os.makedirs(folder, exist_ok=True)
    returnlist = []
    
    for file in file_list:
      returnlist.append(self.download_file(file, folder, check_local=True))
    return tuple(returnlist)

  def checkFolder(self, name):
    assert isinstance(name, str)
    return name in self.folders.keys()

  def createFolder(self, name, exist_ok=True):
    if self.checkFolder(name):
      if exist_ok:
        return True
      raise Exception("Folder {} already exists".format(name))
    self.drive.CreateFile({
        "title" : name,
        "mimeType" : "application/vnd.google-apps.folder",
        "parents" : [{"id" : self.database_gid}]
    }).Upload()
  def move_file(self, file_obj, newFolder):
    assert isinstance(newFolder, str)
    assert self.checkFolder(newFolder)
    files = self.drive.auth.service.files()
    file = files.get(fileId=file_obj['id'], fields ='parents').execute()
    prev_parents = ','.join(p['id'] for p in file.get('parents'))
    file = files.update(
        fileId = file_obj['id'],
        addParents = self.folders[newFolder],
        removeParents = prev_parents,
        fields = 'id, parents',
    ).execute()
    return file
  def trash(self, file_obj):
    self.move_file(file_obj, "TRASH")
    print("Sent {} to trash".format(file_obj['title']))

auth.authenticate_user() # Google auth stuff, make sure to sign in with your ucsb account
gauth = GoogleAuth() # Google auth stuff
gauth.credentials = GoogleCredentials.get_application_default() # Google auth stuff
drive = GoogleDrive(gauth) # Google auth stuff

DOWNLOAD_DATABASE = "1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC"
UPLOAD_DATABASE = "116Y2o-ckMyEb8eGuMwELS_aR5d_t-QOK"
download_database = GoogleDriveDatabase(drive, DOWNLOAD_DATABASE)
upload_database = GoogleDriveDatabase(drive,UPLOAD_DATABASE)
CHARACTERS = ("A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y")

26 folders loaded
25 folders loaded
0 folders loaded


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime
import numpy as np
from PIL import Image


import uuid
import os
import time


def getImagesInFolder(absolutePath):
    assert isinstance(absolutePath, str)
    image_names = [os.path.join(absolutePath, img) for img in os.listdir(absolutePath) if img.endswith(".jpg") or img.endswith(".jpeg") or img.endswith(".png")]
    return image_names

def pixelDataBot(IMAGE_NAME, output_directory):
  # image = DOWNLOAD_DATABASE.download_file(IMAGE_NAME, CHARACTER, check_local = TRUE)
  img = Image.open(IMAGE_NAME).convert('L')
  img_resized = img.resize((720, 480))
  img_resized.save(IMAGE_NAME)
  img_list = np.array(img_resized.getdata()).tolist()
  json_name = IMAGE_NAME.split(".")[0] + ".json"
  json_path = os.path.join(output_directory, json_name)
  json_file = open(json_path, "w")
  json.dump(img_list, json_file)
  json_file.close()
  return json_path

def pixelDataBotGivenPath(input_directory, output_directory):
  images = os.listdir(input_directory)
  os.makedirs(output_directory, exist_ok=True)
  output = []
  for image in images:
    output.append(pixelDataBot(os.path.join(input_directory, image), output_directory))
  return output


    
def download_characters(): # Only downloads characters
  BASE_DIR = "images/"
  results = {}
  for c in CHARACTERS:
    results[c] = download_database.download(c, os.path.join(BASE_DIR, c + "/"))
    print("{} pictures downloaded for {}".format(len(results[c]), c))
  return results
    
def DownloadCharacterAndUploadJson(character,max_count=None):
  strip_name = lambda x: x.split("/")[-1].split(".")[0] # Function to remove extensions (i.e. .json .jpg) from filenames
  character_jsons_original = upload_database.getFiles(character) # All .json files created for this character
  character_jsons_names = [strip_name(x['title']) for x in character_jsons_original] # Name of all .json files already created for this character
  picture_references = download_database.getFiles(character) # All current pictures for this character
  picture_reference_names = [strip_name(x['title']) for x in picture_references] # Name of all pictures
  already_analyzed_names = set() # Names of all already analyzed items
  pictures_to_analyze = [] # Picture that yet do not have a json file
  BASE_FOLDER = "images/" # Directory where all images are
  character_folder = os.path.join(BASE_FOLDER, character + "/") # Folder in which the images for this character are in (i.e. images/A)
  os.makedirs(character_folder, exist_ok=True) # Make the character folder if it doesn't exist
  print("There are {} images for character {}".format(len(picture_references), character))
  print("There are {} json files already made for character {}".format(len(character_jsons_names), character))
  # for json_original in character_jsons_original: # For every json file on the google drive...
  #   if strip_name(json_original['title']) not in picture_reference_names: # ...if the json file is not in the list of pictures...
  #     print("{} deleted because no image was found to relate to it".format(json_original['title']))
  #     json_original.Delete() # ...then delete the file

  for picture in picture_references: # For every picture on google drive...
    if max_count != None:
      max_count -= 1
      if max_count < 0:
        break
    if strip_name(picture['title']) in character_jsons_names: # ...if that picture already has a json uploaded...
      already_analyzed_names.add(download_database.download_file_name(picture['title'])) # ...add its name to the list of already analyzed pictures
      continue
    else:
      pictures_to_analyze.append(download_database.download_file(picture, character_folder, check_local=True)) # ...else download it
  

  local_duplicates = [os.remove(os.path.join(character_folder, x)) for x in os.listdir(character_folder) if x in already_analyzed_names] # if a picture is in the already analyzed list and exists in the local directory, delete it so that openpose doesn't waste time running it
  print("{} local duplicates deleted".format(len(local_duplicates)))
  if len(os.listdir(character_folder)) == 0:
    print("{} has no images to process".format(character))
    return

  BASE_IMAGES_DIR = "/content/images/"
  BASE_JSON_DIR = "/content/json/"
  json_created = pixelDataBotGivenPath(os.path.join(BASE_IMAGES_DIR, character), os.path.join(BASE_JSON_DIR, character)) # runs pixelDataBot on all images in a given path and outputs each image to an output path
  for json_file in json_created: # For every local json file created...
    upload_database.upload(json_file, character, "json") #...upload it to google drive


while True:
  for c in CHARACTERS:
    DownloadCharacterAndUploadJson(c)
  time.sleep(60 * 10)

There are 540 images for character A
There are 0 json files already made for character A
downloaded images/A/A_A_03_30_2020_19_30_25_9.jpg
downloaded images/A/A_A_03_30_2020_19_30_25_8.jpg
downloaded images/A/A_A_03_30_2020_19_30_25_7.jpg
downloaded images/A/A_A_03_30_2020_19_30_25_6.jpg
downloaded images/A/A_A_03_30_2020_19_30_25_5.jpg
0 local duplicates deleted
uploaded and deleted /content/images/A/A_A_03_30_2020_19_30_25_9.json
uploaded and deleted /content/images/A/A_A_03_30_2020_19_30_25_5.json
uploaded and deleted /content/images/A/A_A_03_30_2020_19_30_25_6.json
uploaded and deleted /content/images/A/A_A_03_30_2020_19_30_25_7.json
uploaded and deleted /content/images/A/A_A_03_30_2020_19_30_25_8.json
There are 541 images for character B
There are 0 json files already made for character B
downloaded images/B/B_B_04_04_2020_23_34_22_19.jpg
downloaded images/B/B_B_04_04_2020_23_34_22_18.jpg


KeyboardInterrupt: ignored